# Cassava Leaf Disease Classification Submission

In [1]:
import numpy as np
import pandas as pd

import cv2

from tensorflow.keras import preprocessing
from tensorflow.keras.models import load_model

In [2]:
# constants

IMAGE_SIZE = [512, 512]
MODEL_DIR = '../input/cassava-leaf-disease-classification-model/model.h5'
TEST_DIR = '../input/cassava-leaf-disease-classification/sample_submission.csv'
TEST_IMG_DIR = '../input/cassava-leaf-disease-classification/test_images/'

In [3]:
# data preparation

test_df = pd.read_csv(TEST_DIR)
test_df['label'] = test_df['label'].astype('str')

test_data_gen = preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = test_data_gen.flow_from_dataframe(
    test_df,
    directory=TEST_IMG_DIR,
    x_col='image_id',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='sparse'
)

Found 1 validated image filenames belonging to 1 classes.


In [4]:
# loading the model trained using tpu

model = load_model(MODEL_DIR)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 22,065,701
Trainable params: 22,031,269
Non-trainable params: 34,432
_________________________________________________________________


In [5]:
# prediction

predictions = model.predict(test_gen)
pred_labels = [] # [1, 2, 0, 4, 2, ...]

for pred in predictions:
    pred_labels.append(np.argmax(pred))
    
pred_labels

[4]

In [6]:
# submission

submission = pd.DataFrame({'image_id': test_df.image_id, 'label': pred_labels})
submission.to_csv('submission.csv', index=False) 
submission.head()

,image_id,label
0,2216849948.jpg,4


---